In [ ]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics as crf_metrics
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import wordpunct_tokenize
import nltk
from sklearn.model_selection import train_test_split
import random
from collections import Counter
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!wget dataset_ozon.tsv https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv

--2023-03-23 16:49:29--  http://dataset_ozon.tsv/
Resolving dataset_ozon.tsv (dataset_ozon.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘dataset_ozon.tsv’
--2023-03-23 16:49:29--  https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9640627 (9.2M) [text/plain]
Saving to: ‘dataset_ozon.tsv’

dataset_ozon.tsv    100%[===================>]   9.19M  --.-KB/s    in 0.1s    

2023-03-23 16:49:30 (63.9 MB/s) - ‘dataset_ozon.tsv’ saved [9640627/9640627]

FINISHED --2023-03-23 16:49:30--
Total wall clock time: 1.1s
Downloaded: 1 files, 9.2M in 0.1s (63.9 MB/s)


In [ ]:
ds = pd.read_csv('/content/dataset_ozon.tsv', sep = '\t')
ds = ds.drop(columns = ['Unnamed: 0'])

In [ ]:
ds

,name,brand
0,"Игровая консоль PlayStation 5, белый",PlayStation
1,Золотой Шелк Маска для волос гиалуроновая кисл...,Золотой Шелк
2,"Игровая консоль Microsoft Xbox Series X, черный",Microsoft
3,L'Oreal Paris Infaillible 24H Fresh Wear Пудра...,L'Oreal Paris
4,Maybelline New York Lifter Gloss Блеск для губ...,Maybelline New York
...,...,...
81774,"Erborian BB Family BB карандаш, светлый, 3 г",Erborian
81775,Комплект модулей сменных фильтрующих Аквафор Р...,Аквафор
81776,Набор менструальных чаш Satisfyer Feel secure ...,Satisfyer
81777,28048 Подстилка для надувных и каркасных бассе...,Intex


In [ ]:
def make_tags(n, b):
    global tags
    ner_tags_id = []
    name = wordpunct_tokenize(n)
    brand = wordpunct_tokenize(b)
    results=[]
    sll=len(brand)
    for ind in (i for i,e in enumerate(name) if e==brand[0]):
        if name[ind:ind+sll]==brand:
            results.append((ind,ind+sll-1))
    if results != []:
      ner_tags_labels = ['O'] * len(name)  
      ner_tags_id = [0] * len(name)     
      for result in results:
          if result[0] == result[1]:
              ner_tags_labels[result[0]] = 'B-BRAND'
              ner_tags_id[result[0]] = 1
          else:
            ner_tags_labels[result[0]] = 'B-BRAND'
            ner_tags_id[result[0]] = 1
            for j in range(result[0]+1, result[1]+1):
              ner_tags_labels[j] = 'I-BRAND'
              ner_tags_id[j] = 2

      tags.append({'tokens': name, 'ner_tags_labels': ner_tags_labels})

In [ ]:
tags = []
ds.progress_apply(lambda x: make_tags(x['name'], x['brand']), axis = 1)

100%|██████████| 81779/81779 [00:04<00:00, 17563.50it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
81774    None
81775    None
81776    None
81777    None
81778    None
Length: 81779, dtype: object

# Формируем данные для CRF

In [ ]:
full_list = []
for dictionary in tqdm(tags):
    lil_list = []
    for idx, element in enumerate(dictionary['tokens']):
        lil_list.append({
            'text': element,
            'bio-tag': dictionary['ner_tags_labels'][idx]
        })
    full_list.append(lil_list)

100%|██████████| 81527/81527 [00:01<00:00, 60873.95it/s]


In [ ]:
random.shuffle(full_list)

In [ ]:
train_sents, test_sents = train_test_split(full_list)

In [ ]:
train_sents[0]

[{'text': 'Planta', 'bio-tag': 'B-BRAND'},
 {'text': 'Массажер', 'bio-tag': 'O'},
 {'text': 'для', 'bio-tag': 'O'},
 {'text': 'ног', 'bio-tag': 'O'},
 {'text': 'MF', 'bio-tag': 'O'},
 {'text': '-', 'bio-tag': 'O'},
 {'text': '6B', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'Шиацу', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': '3', 'bio-tag': 'O'},
 {'text': 'уровня', 'bio-tag': 'O'},
 {'text': 'компрессии', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'подогрев', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': '3', 'bio-tag': 'O'},
 {'text': 'скорости', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'панель', 'bio-tag': 'O'},
 {'text': 'управления', 'bio-tag': 'O'},
 {'text': ',', 'bio-tag': 'O'},
 {'text': 'таймер', 'bio-tag': 'O'}]

In [ ]:
test_sents[0]

[{'text': 'Серьги', 'bio-tag': 'O'},
 {'text': 'LUMINA', 'bio-tag': 'B-BRAND'},
 {'text': 'ART', 'bio-tag': 'I-BRAND'},
 {'text': 'JEWELSDIN77', 'bio-tag': 'I-BRAND'}]

In [ ]:
def word2features(sent, i):
    word = sent[i]['text']
    
    features = {
        'text': word
    }
    if i > 0:
        word1 = sent[i-1]['text']
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]['text']
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word['bio-tag'] for word in sent]

def sent2tokens(sent):
    return [word['text'] for word in sent]

In [ ]:
X_train = [sent2features(text) for text in train_sents]
y_train = [sent2labels(text) for text in train_sents]

X_test = [sent2features(text) for text in test_sents]
y_test = [sent2labels(text) for text in test_sents]

In [ ]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
y_pred  = crf.predict(X_test)

In [ ]:
labels = set()
for y_big in y_train:
  for y_small in y_big:
    labels.add(y_small)

labels = list(labels)

In [ ]:
crf_metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.9843246584347566

In [ ]:
crf_metrics.flat_precision_score(y_test, y_pred, average='weighted', labels=labels)

0.9843790240393467

In [ ]:
crf_metrics.flat_recall_score(y_test, y_pred, average='weighted', labels=labels)

0.9845698840935615